In [20]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import fsolve, minimize

###  Black-Schole functions from previous videos

In [21]:
def call_price(sigma, S, K, r, t):
    d1 = np.multiply( 1. / sigma * np.divide(1., np.sqrt(t)),
        np.log(S/K) + (r + sigma**2 / 2.) * t  )
    d2 = d1 - sigma * np.sqrt(t)
    
    C = np.multiply(S, norm.cdf(d1)) - \
        np.multiply(norm.cdf(d2) * K, np.exp(-r * t))
    return C

def put_price(sigma, S, K, r, t):
    d1 = np.multiply( 1. / sigma * np.divide(1., np.sqrt(t)),
        np.log(S/K) + (r + sigma**2 / 2.) * t  )
    d2 = d1 - sigma * np.sqrt(t)
    
    P = -np.multiply(S, norm.cdf(-d1)) + \
        np.multiply(norm.cdf(-d2) * K, np.exp(-r * t))
    return P

#  Scenario 1:  We know the price of both legs
If we have the price of both legs, but not the price of the stock or the implied volatility, we can solve a system of equations based on the Black-Scholes model to estimate those values.

$$C(S, \sigma | K, r, t) = \mbox{Black-Scholes price of a call}$$

$S$ = price of the stock<br>
$\sigma$ = implied volatility<br>
$K_1$  Strike price of first leg<br>
$K_2$ = strike price of second leg<br>
$C_1$ = Price of first leg<br>
$C_2$ = price of second leg<br>
$r$ = risk-free rate<br>
$t$ = time to expiration

$$
\begin{array}{ll}
C(S, \sigma | K_1, r, t) - C_1 & = 0 \\
C(S, \sigma | K_2, r, t) - C_2 & = 0
\end{array}
$$

We can use the function fsolve from the scipy.optimize package to solve this.

Another way of solving this is to use something similar to a least squares fit.
Consider the function

$$
J = (C(S, \sigma | K_1, r, t) - C_1)^2 + (C(S, \sigma | K_2, r, t) - C_2)^2
$$


This has the advantage we can set additional constraints for the minimize function.
<img src="aapl_theta.png" width="40%">

<i>fsolve</i> solves systems of the form 
$$f(x) = 0$$
So we need to code our function to be equatl to zero.

In [22]:
def equation_system_1(x, C_1, C_2, K_1, K_2, r, t):
    S = x[0]
    sigma = x[1]
    
    res = np.zeros(2)
    
    res[0] = call_price(sigma, S, K_1, r, t) - C_1
    res[1] = call_price(sigma, S, K_2, r, t) - C_2
    
    return res

#  Option values from the chain.  Time and interest rates
t = 13.0 / 365.0
r = 0.01

#  Strikes
K_1 = 130
K_2 = 135

#  Option prices
C_1 = 2.57
C_2 = 1.59

#  Additional values to pass to our function
args = (C_1, C_2, K_1, K_2, r, t)

x = np.array([100, 1.0])
x = fsolve(equation_system_1, x, args = args)
print(x)

[119.2102042    0.70420356]


##  Solution via Least Squares Minimization

The constraint funcion assumes:
$$f(x) \geq C$$
but because the function must return the residual, we code it as,
$$f(x) - C \geq 0.$$

In [23]:
def objective(x, C_1, C_2, K_1, K_2, r, t):
    S = x[0]
    sigma = x[1]
    
    res = np.zeros(2)
    
    #  Calculate the error of each leg
    res[0] = call_price(sigma, S, K_1, r, t) - C_1
    res[1] = call_price(sigma, S, K_2, r, t) - C_2
    
    #  Construct and return sum of squared errors
    J = res[0]**2 + res[1]**2
    
    return J

def constraint(x):
    C = np.zeros(2)
    
    #  I used 0.01 rather than 0 for the stock and vol minimum value to avoid potential divide-by-zero issues
    C[0] = x[0] - 0.01
    C[1] = x[1] - 0.01
    
    return C

#  Set constraint dictionary
cons = {'type':'ineq', 'fun': constraint}

#  Our initial guess of the price and vol.
x = np.array([100, 0.3])

#  Solve the optimization problem
x = minimize(objective, x, args = args, constraints = cons)
print(x)

     fun: 1.587883078763436e-08
     jac: array([3.91922746e-05, 1.29724706e-03])
 message: 'Optimization terminated successfully.'
    nfev: 104
     nit: 24
    njev: 24
  status: 0
 success: True
       x: array([119.20624468,   0.70434781])


#  Scenario 2:  We know the Price of the spread of noteach leg
Here, we will assume we know the price of the spread, but not the price of each leg.  In this case, we will have to know the volatility to come up with an answer.

$$\mbox{Spread Price} = C_1 - C_2$$

So, we want to solve,

$$ (C(S | \sigma, K_1, r, t) - C(S | \sigma, K_2, r, t)) - \mbox{Spread Price} = 0$$

In [24]:
#  We need to know the vol.  We are using the values from above so we can check our ansqwer.
sigma = 0.704

#  Likewise, we are using the spread price from above for debugging purposes.  In this scenario, we would not really know the price of each leg.
spread = C_1 - C_2

def spread_system_2(x, spread,  K_1, K_2, sigma, r, t):
    S = x
    
    #  Construct the equation as noted above
    C_1 = call_price(sigma, S, K_1, r, t)
    C_2 = call_price(sigma, S, K_2, r, t)
    
    return C_1 - C_2 - spread

args = (spread,  K_1, K_2, sigma, r, t)
S = fsolve(spread_system_2, 100, args = args)
print(S)

[119.21350149]
